In [1]:
import os 
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

2025-04-17 00:06:37.802970: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744848398.049583      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744848398.127203      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Load the CSV file
data_path = '/kaggle/input/ham10000-preprocessed-dataset/balanced and standardized images meta for custom.csv'
image_folder = '/kaggle/input/ham10000-preprocessed-dataset/balanced_and_standardized_images/balanced_and_standardized_images'
df = pd.read_csv(data_path)

# Define parameters
IMG_SIZE = 96
BATCH_SIZE = 32
LEARNING_RATE = 0.0001
DROPOUT_RATE = 0.5
EPOCHS = 20

# Get unique classes and create label mapping
classes = df['dx'].unique()
class_to_idx = {c: i for i, c in enumerate(classes)}
df['label'] = df['dx'].map(class_to_idx)

# Extract labels and encode them
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["dx"])  # Encode the 'dx' column



In [3]:
# Train-test split
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

In [4]:
# Image Data Generators with Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% of train set will be used as validation
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, directory=image_folder, x_col='image_id', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', subset='training'
)

val_generator = train_datagen.flow_from_dataframe(
    train_df, directory=image_folder, x_col='image_id', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    test_df, directory=image_folder, x_col='image_id', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False
)

Found 30039 validated image filenames belonging to 7 classes.
Found 7509 validated image filenames belonging to 7 classes.
Found 9387 validated image filenames belonging to 7 classes.


In [5]:
# Build Xception Model
base_model = Xception(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False  # Freeze base model layers

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(DROPOUT_RATE)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(DROPOUT_RATE)(x)
output = Dense(len(label_encoder.classes_), activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[early_stop]
)

2025-04-17 00:15:39.008294: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


939/939 ━━━━━━━━━━━━━━━━━━━━ 1142s 1s/step - accuracy: 0.2262 - loss: 1.9640 - val_accuracy: 0.3967 - val_loss: 1.5966
Epoch 2/20
939/939 ━━━━━━━━━━━━━━━━━━━━ 759s 806ms/step - accuracy: 0.3503 - loss: 1.6553 - val_accuracy: 0.4372 - val_loss: 1.4975
Epoch 3/20
939/939 ━━━━━━━━━━━━━━━━━━━━ 809s 813ms/step - accuracy: 0.3765 - loss: 1.5930 - val_accuracy: 0.4440 - val_loss: 1.4506
Epoch 4/20
939/939 ━━━━━━━━━━━━━━━━━━━━ 754s 801ms/step - accuracy: 0.3907 - loss: 1.5407 - val_accuracy: 0.4600 - val_loss: 1.4165
Epoch 5/20
939/939 ━━━━━━━━━━━━━━━━━━━━ 801s 801ms/step - accuracy: 0.4058 - loss: 1.5177 - val_accuracy: 0.4584 - val_loss: 1.3920
Epoch 6/20
939/939 ━━━━━━━━━━━━━━━━━━━━ 755s 802ms/step - accuracy: 0.4148 - loss: 1.4908 - val_accuracy: 0.4773 - val_loss: 1.3591
Epoch 7/20
939/939 ━━━━━━━━━━━━━━━━━━━━ 751s 797ms/step - accuracy: 0.4294 - loss: 1.4564 - val_accuracy: 0.4866 - val_loss: 1.3567
Epoch 8/20
939/939 ━━━━━━━━━━━━━━━━━━━━ 805s 856ms/step - accuracy: 0.4291 - loss: 1.4558

In [14]:
# Plotting
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_model_training_curve(history):
    fig = make_subplots(rows=1, cols=2, subplot_titles=['Model Accuracy', 'Model Loss'])
    fig.add_trace(
        go.Scatter(
            y=history.history['accuracy'], 
            name='train_acc'), 
        row=1, col=1)
    fig.add_trace(
        go.Scatter(
            y=history.history['val_accuracy'], 
            name='val_acc'), 
        row=1, col=1)
    fig.add_trace(
        go.Scatter(
            y=history.history['loss'], 
            name='train_loss'), 
        row=1, col=2)
    fig.add_trace(
        go.Scatter(
            y=history.history['val_loss'], 
            name='val_loss'), 
        row=1, col=2)
    fig.show()

plot_model_training_curve(history)



In [12]:
# Test function
def evaluate_model():
    test_loss, test_acc = model.evaluate(test_generator)
    print(f'\nTest Accuracy: {test_acc * 100:.2f}%')

# Run evaluation
evaluate_model()

294/294 ━━━━━━━━━━━━━━━━━━━━ 185s 630ms/step - accuracy: 0.4697 - loss: 1.3520

Test Accuracy: 46.66%


In [13]:
# Saving the model
model.save('/kaggle/working/xceptionnet 80-20.keras')
model.save_weights('/kaggle/working/xceptionnet 80-20.weights.h5')